<a href="https://colab.research.google.com/github/HuyenNguyenHelen/LING-5412/blob/main/Assignment1_NB_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import os
import tarfile
import re
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_validate


In [2]:
my_tar = tarfile.open('/content/lingspam_public.tar.gz')
my_tar.extractall('/content/') 
my_tar.close()
train_path = '/content/lingspam_public/lemm_stop/part1'  # for training      #spams: spmsg*.txt
test_path = '/content/lingspam_public/lemm_stop/part10'   # for testing

In [3]:
def to_dict (path):
  data_dict = dict()
  data_dict[1] = []
  data_dict[0] = []
  for file in os.listdir(path):  
    doc = open (path + '/'+ file, 'r')
    if 'spmsg' in file:
      data_dict[1].append(doc.read())
    else:
      data_dict[0].append(doc.read())
  print ('number of spams: {}'.format(len(data_dict[1])))
  print ('number of not_spams: {}'.format(len(data_dict[0])))
  n_docs = len(os.listdir(path))
  return data_dict, n_docs

print('training set:')
training, n_docs_train = to_dict (train_path)
print('number of doc: {}'.format(n_docs_train))

print('\ntesting set:')
testing, n_docs_test = to_dict (test_path)
print('number of doc: {}'.format(n_docs_test))


training set:
number of spams: 48
number of not_spams: 241
number of doc: 289

testing set:
number of spams: 49
number of not_spams: 242
number of doc: 291


In [14]:
def tokenizer (doc):
  doc = doc.lower()
  doc_clean = doc.lower()
  return re.split("\W+", doc)

def to_bow (data_dict):
  bows = {}
  bows[0], bows[1] = [], []
  for doc in data_dict[0]:
    bows[0].extend(tokenizer(doc))
  for doc in data_dict[1]:
    bows[1].extend(tokenizer(doc))
  return bows

def train_NB (training):
  bows = to_bow (training)
  set_V = set(bows[1] + bows[0])
  print(len(bows[1] + bows[0]), len(set_V))
  results={}
  for c in training.keys():
    results[c]={}
    loglikelihood_c = {}
    n_c = len(training[c])
    logprior_c = np.log(n_c/n_docs_train)
    print('log prior class {}: {}'. format(c, logprior_c))
    count_w_c = {}
    for doc in training[c]:
      for token in tokenizer(doc):
        count_w_c[token] = count_w_c.get(token, 0)+1
    print ('length of word count {}: {}'.format(c, len(count_w_c.keys())))
    for w in set_V:
      if w in count_w_c.keys():
        count_w = count_w_c[w]
      else:
        count_w = 0
      loglikelihood_w = np.log(count_w+1/(sum(count_w_c.values())+len(set_V)))
      loglikelihood_c[w] = loglikelihood_w

    results[c]['likelihood_w']=loglikelihood_c
    print('sum word count in {}: {}'. format(c, sum(count_w_c.values())))
    results[c]['logprior_c']=logprior_c
    results[c]['set_V']=set_V
    print('length of set V: ', len(set_V))
  #print(results[1]['likelihood_w'])
  return results

train_result = train_NB (training)

68474 11315
log prior class 1: -1.7952256772045412
length of word count 1: 2972
sum word count in 1: 21015
length of set V:  11315
log prior class 0: -0.18162975462177716
length of word count 0: 10031
sum word count in 0: 47459
length of set V:  11315


In [ ]:
train_result[1].keys()

dict_keys(['likelihood_w', 'logprior_c', 'set_V'])

In [20]:
X_train = training[0]+ training[1]
y_train = [0]*len(training[0]) + [1]*len(training[1])

X_test = testing[0]+ testing[1]
y_true = [0]*len(testing[0]) + [1]*len(testing[1])
def test_NB (testing):
  prediction = []
  X_test = testing[0]+ testing[1]
  for doc in X_test:
    spam_loglikelihoods, nonspam_loglikelihoods = [], []
    spam_score=0
    nonspam_score = 0
    for w in tokenizer (doc):
      if w not in train_result[0]['set_V']: continue
      if w in train_result[1]['likelihood_w'].keys(): 
        spam_score += train_result[1]['likelihood_w'][w] 
      else:
        spam_score += 0
      if w in train_result[0]['likelihood_w'].keys(): 
        nonspam_score += train_result[0]['likelihood_w'][w] 
      else:
        nonspam_score += 0
    
    #   spam_loglikelihoods.append(train_result[1]['likelihood_w'][w] if w in train_result[1]['likelihood_w'].keys() else 0)
    #   nonspam_loglikelihoods.append(train_result[0]['likelihood_w'][w] if w in train_result[0]['likelihood_w'].keys() else 0)
    # spam_score += sum(spam_loglikelihoods)
    # nonspam_score += sum(nonspam_loglikelihoods)

    spam_logprior = train_result[1]['logprior_c']
    nonspam_logprior = train_result[0]['logprior_c']
    spam_score += spam_logprior
    nonspam_score += nonspam_logprior
    if spam_score > nonspam_score:
      prediction.append(1)
    else:
      prediction.append(0)
  return prediction

y_pred = test_NB (testing)
# print(y_true)
# print(y_pred)

print('accuracy on training set: {}'.format(sklearn.metrics.accuracy_score(y_train, test_NB(training))))
print(classification_report(y_train, test_NB(training)))

print('\naccuracy on test set: {}'.format(sklearn.metrics.accuracy_score(y_true, y_pred)))
print(classification_report(y_true, y_pred))

accuracy on training set: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       241
           1       1.00      1.00      1.00        48

    accuracy                           1.00       289
   macro avg       1.00      1.00      1.00       289
weighted avg       1.00      1.00      1.00       289


accuracy on test set: 0.9037800687285223
              precision    recall  f1-score   support

           0       0.90      1.00      0.95       242
           1       0.96      0.45      0.61        49

    accuracy                           0.90       291
   macro avg       0.93      0.72      0.78       291
weighted avg       0.91      0.90      0.89       291

